In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install nlppreprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
import string
from nlppreprocess import NLP
import pickle
# splitting data
from sklearn.model_selection import train_test_split
# Features Extraction
from sklearn.feature_extraction.text import CountVectorizer
# Features Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC

from sklearn import preprocessing

from sklearn.svm import SVC

# # Features Extraction word2vec
# from gensim.models import Word2Vec

# # Dimension reduction
# from sklearn.decomposition import PCA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import io
df_file = pd.read_csv(io.BytesIO(uploaded['train_test.csv']))
df_file.dropna(inplace=True)
print(len(df_file))
df_file.head()

31014


,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [ ]:
df_file = df_file.sample(frac=1).reset_index(drop=True) # shuffel
df_file = df_file[:25000]
print(len(df_file))
df_file.head()

25000


,textID,text,sentiment
0,7f1d2525ba,"Should work fine, as far as I know. *shrug* ...",neutral
1,e0cd486f0f,- I always appreciate your 'musings.',positive
2,d0d3c6f298,cool i wear black most of the time when i go out,neutral
3,ecf151daa5,"Not feeling too good at ALL, about to head to ...",negative
4,5ef5adb487,Just got Dailybooth and not sure what to do. I...,negative


In [ ]:


def one_string_lower_conversation(sentence):
    '''
    Argument:
        text as string of words
    return:
        lower of this string
    '''
    return sentence.lower()
  
def all_string_lower_conversation(text_list):
    '''
    Argument:
        list of strings and each of these strings does contain some of words
    return:
        lower each string in this list
    '''
    text_list = [one_string_lower_conversation(sentence) for sentence in text_list]
    return text_list


def one_string_remove_punctuation(sentence):
    '''
    Argument:
        string of words
    reutrn:
        string without punctuation like [.!?] and others
    '''
    sentence = re.sub("""(@\w+[1-9:'\/"]? |http:\/\/\w+.\w+.\w+\/)""", '', sentence)
    sentence = sentence.split(' ')
    
    strs = ''
    punctuations = string.punctuation
    for word in sentence:
        word = re.sub('[^\w\s+]',' ',word)
        strs += word + ' '
    translator = str.maketrans('', '', punctuations)
    strs.translate(translator)
    return strs

def all_strings_remove_punctuation(text_list):
    '''
    Argument:
        list of strings 
    reutrn:
        list of strings without punctuation like [.!?] and others
    '''
    text_list = [one_string_remove_punctuation(sentence) for sentence in text_list]
    return text_list


def one_string_tokenization(sentence):
    '''
    Argument:
        String of words
    return:
        list of words
    '''
    sentence = word_tokenize(sentence)
    return sentence


def all_string_tokenization(text_list):
    '''
    Argument:
        list of Strings
    return:
        list of strings and every string is list of words
    '''
    text_list = [one_string_tokenization(sentence) for sentence in text_list]
    return text_list


def one_string_Lemmatizing(sentence):
    '''
    Argument:
        String of words
    return:
        list of words with Lemmatizing
    '''
    sentence = one_string_tokenization(sentence)
    lemmatizer = WordNetLemmatizer()
    sentence = [lemmatizer.lemmatize(word) for word in sentence]
    return sentence

def all_string_Lemmatizing(text_list):
    '''
    Argument:
        list of strings
    return:
        list of strings with steming which the root of the word in each string
    '''
    text_list = [one_string_Lemmatizing(sentence) for sentence in text_list]
    return text_list


def one_string_stop_words(sentence):
    '''
    Argument:
        string of words
    return:
        remove stop words from this string like this, did
        but other words like not, no dont remove
	'''
    stop_words = NLP().stopword_list
    sentence = sentence.split(' ')
    updated_sentence = ''
    for word in sentence:
    	if word not in stop_words:
    		updated_sentence += word + ' '
    return updated_sentence






def all_string_stop_words(text_list):
  '''
  Argument:
      list of string
  return:
      list of string without stop words
  '''
      
  text_list = [one_string_stop_words(sentence) for sentence in text_list]
  return text_list

def one_string_un_tokenization(sentence):
    '''
    Argument:
        list of words
    return:
        string of words
    '''
    sentence = " ".join(sentence)
    return sentence
    
def all_string_un_tokenization(text_list):
    '''
    Argument:
        list of words
    return:
        string of words
    '''
    text_list = [one_string_un_tokenization(sentence) for sentence in text_list]
    return text_list

def english_pip_line(text_list):
    text_list = all_string_lower_conversation(text_list)
    text_list = all_strings_remove_punctuation(text_list)
    text_list = all_string_stop_words(text_list)
    return text_list



In [ ]:
print(np.unique(df_file['sentiment']))
df_file.columns

['negative' 'neutral' 'positive']


Index(['textID', 'text', 'sentiment'], dtype='object')

In [ ]:
text_list = list(df_file['text'])
text_list = english_pip_line(text_list)
len(text_list)


25000

In [ ]:
X = text_list
y = df_file['sentiment']

In [ ]:
def count_vectorize(df):
    '''
    Argumen:
        df dataframe of multiple reviews
    return:
        Train & test arrays that can fir to the model
    '''
# I fit the vector to all of the data
    vectorizer = CountVectorizer()
    vectorize = vectorizer.fit(df) 
    return vectorizer

In [ ]:
count_vectorizer = count_vectorize(X)
pickle.dump(count_vectorizer, open("count_vectorizer_25000.pickle", "wb"))
files.download('count_vectorizer_25000.pickle')
count_vectorizer = pickle.load(open("count_vectorizer_25000.pickle", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# convert y from text to label encoder and we can see all entiment labels now
LabelEncoder = preprocessing.LabelEncoder()
LabelEncoder.fit(y)
list(LabelEncoder.classes_)
y = LabelEncoder.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print("Our training data now are: " + str(len(X_train))  + " tweet")
print("Our testing data now are: " + str(len(X_test))  + " tweet")
print("Our training data now are: " + str(len(y_train))  + " labels")
print("Our testing data now are: " + str(len(y_test))  + " labels")

Our training data now are: 20000 tweet
Our testing data now are: 5000 tweet
Our training data now are: 20000 labels
Our testing data now are: 5000 labels


In [ ]:

training_data = count_vectorizer.transform(X_train)
training_data  = training_data.toarray()
testing_data = count_vectorizer.transform(X_test)
testing_data  = testing_data.toarray()

In [ ]:
# first shape is the data itself and second shape is the BOW in our data
print("Our new vectorized data: " + str(training_data.shape))
print("Our new vectorized data: " + str(testing_data.shape)) 
print("The first 2 review after transform: \n", testing_data[:2])

Our new vectorized data: (20000, 24673)
Our new vectorized data: (5000, 24673)
The first 2 review after transform: 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
clf_MultinomialNB = MultinomialNB()
model = clf_MultinomialNB.fit(training_data, y_train)
predict = model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
print("Evalution Matrix of training data is \n", confusion_matrix(y_train, predict))
predict = model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))
print("Evalution Matrix of training data is \n", confusion_matrix(y_test, predict))

F1 score of our training data is:  0.8422
Evalution Matrix of training data is 
 [[4602  951  191]
 [ 446 7007  584]
 [ 168  816 5235]]
F1 score of our testing data is:  0.6612
Evalution Matrix of training data is 
 [[ 862  482   77]
 [ 321 1383  315]
 [  62  437 1061]]


In [ ]:
pickle.dump(model, open("MultinomialNB_63%_25000.pickle", "wb"))
files.download('MultinomialNB_63%_25000.pickle')
count_vectorizer = pickle.load(open("MultinomialNB_63%_25000.pickle", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', solver='liblinear')
logistic_model = clf_LogisticRegression.fit(training_data, y_train)
predict = logistic_model.predict(training_data)
print("F1 score of our testing data is: ", f1_score(y_train, predict, average='micro'))
print("Evalution Matrix of training data is \n", confusion_matrix(y_train, predict))
predict = logistic_model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))
print("Evalution Matrix of training data is \n", confusion_matrix(y_test, predict))

F1 score of our testing data is:  0.88185
Evalution Matrix of training data is 
 [[4906  679  159]
 [ 412 7199  426]
 [ 128  559 5532]]
F1 score of our testing data is:  0.708
Evalution Matrix of training data is 
 [[ 898  463   60]
 [ 265 1502  252]
 [  66  354 1140]]


In [ ]:
pickle.dump(model, open("logistic_model_67%_25000.pickle", "wb"))
# files.download('logistic_model_67%_25000.pickle')
count_vectorizer = pickle.load(open("logistic_model_67%_25000.pickle", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>